# PyTorch Assignment: Multi-Layer Perceptron (MLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: 

### Multi-Layer Perceptrons

The simple logistic regression example we went over in the previous notebook is essentially a one-layer neural network, projecting straight from the input to the output predictions.
While this can be effective for linearly separable data, occasionally a little more complexity is necessary.
Neural networks with additional layers are typically able to learn more complex functions, leading to better performance.
These additional layers (called "hidden" layers) transform the input into one or more intermediate representations before making a final prediction.

In the logistic regression example, the way we performed the transformation was with a fully-connected layer, which consisted of a linear transform (matrix multiply plus a bias).
A neural network consisting of multiple successive fully-connected layers is commonly called a Multi-Layer Perceptron (MLP). 
In the simple MLP below, a 4-d input is projected to a 5-d hidden representation, which is then projected to a single output that is used to make the final prediction.

<img src="Figures/MLP.png" width="300"/>

For the assignment, you will be building a MLP for MNIST.
Mechanically, this is done very similary to our logistic regression example, but instead of going straight to a 10-d vector representing our output predictions, we might first transform to a 500-d vector with a "hidden" layer, then to the output of dimension 10.
Before you do so, however, there's one more important thing to consider.

### Nonlinearities

We typically include nonlinearities between layers of a neural network.
There's a number of reasons to do so.
For one, without anything nonlinear between them, successive linear transforms (fully connected layers) collapse into a single linear transform, which means the model isn't any more expressive than a single layer.
On the other hand, intermediate nonlinearities prevent this collapse, allowing neural networks to approximate more complex functions.

There are a number of nonlinearities commonly used in neural networks, but one of the most popular is the [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)):

\begin{align}
x = \max(0,x)
\end{align}

There are a number of ways to implement this in PyTorch.
We could do it with elementary PyTorch operations:

In [1]:
import torch

x = torch.rand(5, 3)*2 - 1
x_relu_max = torch.max(torch.zeros_like(x),x)

print("x: {}".format(x))
print("x after ReLU with max: {}".format(x_relu_max))

x: tensor([[-0.1652,  0.9928,  0.2487],
        [-0.4176, -0.4108, -0.2655],
        [ 0.2589,  0.7418,  0.8917],
        [ 0.6096, -0.3041,  0.4049],
        [-0.7718, -0.8736, -0.5774]])
x after ReLU with max: tensor([[0.0000, 0.9928, 0.2487],
        [0.0000, 0.0000, 0.0000],
        [0.2589, 0.7418, 0.8917],
        [0.6096, 0.0000, 0.4049],
        [0.0000, 0.0000, 0.0000]])


Of course, PyTorch also has the ReLU implemented, for example in `torch.nn.functional`:

In [2]:
import torch.nn.functional as F

x_relu_F = F.relu(x)

print("x after ReLU with nn.functional: {}".format(x_relu_F))

x after ReLU with nn.functional: tensor([[0.0000, 0.9928, 0.2487],
        [0.0000, 0.0000, 0.0000],
        [0.2589, 0.7418, 0.8917],
        [0.6096, 0.0000, 0.4049],
        [0.0000, 0.0000, 0.0000]])


Same result.

### Assignment

Build a 2-layer MLP for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image (784 dimensions) ->  
fully connected layer (500 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax

Try building the model both with basic PyTorch operations, and then again with more object-oriented higher-level APIs. 
You should get similar results!


*Some hints*:
- Even as we add additional layers, we still only require a single optimizer to learn the parameters.
Just make sure to pass all parameters to it!
- As you'll calculate in the Short Answer, this MLP model has many more parameters than the logisitic regression example, which makes it more challenging to learn.
To get the best performance, you may want to play with the learning rate and increase the number of training epochs.
- Be careful using `torch.nn.CrossEntropyLoss()`. 
If you look at the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#crossentropyloss): you'll see that `torch.nn.CrossEntropyLoss()` combines the softmax operation with the cross-entropy.
This means you need to pass in the logits (predictions pre-softmax) to this loss.
Computing the softmax separately and feeding the result into `torch.nn.CrossEntropyLoss()` will significantly degrade your model's performance!

In [5]:
### YOUR CODE HERE
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms

mnist_train = datasets.MNIST(root='./datasets', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root='./datasets', train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False, pin_memory=True)

W1 = torch.randn(784, 500)/np.sqrt(748)
W1.requires_grad_()
b1 = torch.zeros(500, requires_grad=True)

W2 = torch.randn(500, 10)/np.sqrt(500)
W2.requires_grad_()
b2 = torch.zeros(10, requires_grad=True)

optimizer = torch.optim.SGD([W1,W2,b1,b2], lr=0.1)

for images, labels in train_loader:
    optimizer.zero_grad()
    
    a1 = images.view(-1, 28*28)
    z1 = torch.matmul(a1, W1) + b1
    a2 = F.relu(z1)
    z2 = torch.matmul(a2, W2) + b2
    cross_entropy = F.cross_entropy(z2, labels)
    
    cross_entropy.backward()
    optimizer.step()

correct = 0
total = len(mnist_test)

with torch.no_grad():
    for images, labels in test_loader:
        a1 = images.view(-1, 28*28)
        z1 = torch.matmul(a1, W1) + b1
        a2 = F.relu(z1)
        z2 = torch.matmul(a2, W2) + b2
        
        predictions = torch.argmax(z2, dim=1)
        correct += torch.sum((predictions==labels).float())

# Make sure to print out your accuracy on the test set at the end.
print(f'Test accuracy: {correct/total}')

Test accuracy: 0.926800012588501


---

In [2]:
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [3]:
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))
print(torch.cuda.is_available())

0
NVIDIA GeForce GTX 1650 Ti
True


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [5]:
device

device(type='cuda', index=0)

In [18]:
class MNIST_MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(784, 500)
        self.lin2 = nn.Linear(500, 200)
        self.lin3 = nn.Linear(200, 10)
    
    def forward(self, x):
        z1 = self.lin1(x)
        a2 = F.relu(z1)
        z2 = self.lin2(a2)
        a3 = F.relu(z2)
        z3 = self.lin3(a3)
        return z3

In [7]:
mnist_train = datasets.MNIST(root='./datasets', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root='./datasets', train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False, pin_memory=True)

c:\users\user\appdata\local\programs\python\python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [19]:
model = MNIST_MLP().to(device)
print(f'model is running on GPU: {next(model.parameters()).is_cuda}')

loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
epochs = 10

print('Allocated:', round(torch.cuda.memory_allocated(device=0)/1e+9, 1), 'GB')
for epoch in range(epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        x = images.view(-1, 28*28)
        y = model(x)
        loss = loss_func(y, labels)
        
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch} finishs')

model is running on GPU: True
Allocated: 1.6 GB
Epoch 0 finishs
Epoch 1 finishs
Epoch 2 finishs
Epoch 3 finishs
Epoch 4 finishs
Epoch 5 finishs
Epoch 6 finishs
Epoch 7 finishs
Epoch 8 finishs
Epoch 9 finishs


In [20]:
correct = 0
total = len(mnist_test)

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        x = images.view(-1, 28*28)
        y = model(x)
        
        predictions = torch.argmax(y, dim=1)
        correct += torch.sum((predictions==labels).float())
    
print(f'Test accuracy: {correct/total}')

Test accuracy: 0.9763999581336975


### Short answer
How many trainable parameters does your model have? 
How does this compare to the logisitic regression example?

In [27]:
model

MNIST_MLP(
  (lin1): Linear(in_features=784, out_features=500, bias=True)
  (lin2): Linear(in_features=500, out_features=200, bias=True)
  (lin3): Linear(in_features=200, out_features=10, bias=True)
)

`[Your answer here]`